title: AB Test: Remove video kyc link from flow   
author: Fabio Schmidt-Fischbach  
date: 2020-06-10  
region: EU  
summary: Control - before initiating the KYC call, we have a link to an explainer video (like this one) on youtube. We remove this link in the treatment group. Clicking on this video might take users out of the KYC flow. Main conversion rate of starting the KYC flow to initiating a KYC call increased from 92% to 93% in the treatment. Crucially, the higher number of people initiating the call was not counteracted by a lower % completing the call. Rather, we observe the same % KYCi to KYCc (7day) conversion for both treatment and control. What's the impact of this feature then? Monthly, we currently have roughly 30k users starting the video flow. Increasing the % that start and also finish KYC from 63% to 64% produces an added = 30k x 1% = 300 KYCc per month. Annually, we'd expect an added 3600 KYCc per year and an increase of something close to 1pp in the video flow conversion rate.
tags: kyc, acquire, ab test  

In [1]:
import pandas as pd
import os
import seaborn as sns
from statsmodels.stats.proportion import proportions_ztest
import altair as alt

### 0. A note on why we need AB testing for product development. 

This test started with a UX research driven question. Is it a good thing to have a video link on the KYC video screen allowing users to exit the flow? the data team looking at the correlation between clicking on the link to view the video and progressing with the flow. 

While our UX assessment told us that taking people out of the flow might be detrimental, the correlation suggested the opposite. People who clicked the video were more likely to move on with their KYC journey. 

Why did we move on with the project anyway? Our main interpretation at the time was that very ambitious users would be much more likely to both click on the link and then also to progress. The correlation hence did not only capture the effect of taking the users out of the flow but also partly it told us a story user quality. 

To isolate the effect of the video link, we needed an AB test.


### 1. Setup. 

- Target group: users that do KYC via video (IDNow) since 20th May 2020 and have the current app versions installed. 
- Feature change: Control - before initiating the KYC call, we have a link to an explainer video (like this one) on youtube. We remove this link in the treatment group. 
- Rational: Clicking on this video might take users out of the KYC flow.
- Main KPIs: conversion to KYCi and KYCi to KYCc.


### 2. Summary 

Main conversion rate of starting the KYC flow to initiating a KYC call increased from 92% to 93% in the treatment. 

Crucially, the higher number of people initiating the call was not counteracted by a lower % completing the call. Rather, we observe the same % KYCi to KYCc (7day) conversion for both treatment and control. 

What's the impact of this feature then? 

Monthly, we currently have roughly 30k users starting the video flow. Increasing the % that start and also finish KYC from 63% to 64% produces an added = 30k x 1% = 300 KYCc per month. 

Annually, we'd expect an added 3600 KYCc per year and an increase of something close to 1pp in the video flow conversion rate.




In [2]:
##### Query to pull the data from the dwh.
query = """ 

---get set of users that were part of the experiment for the first time. 
with users as ( 

select user_created, min(platform) as platform, min(os_major) as os_major    
from dbt.zrh_kyc_funnel 
where se_action like 'kyc.video%' and app_version in ('n26-ios_3.45', 'n26-ios_3.46', 'n26-android_3.45') and created >= '2020-05-20'
group by 1 

)

select  case when is_user_in_tg(user_id, 'idnow.show-video-link', 50) = True then 'Control' else 'Treatment' end as treatment, 
		zu.user_id,
		tnc_country_group,
		product_id, 
		kyc_first_initiated, 
		kyc_first_completed, 
		age_group,
		gender, 
		platform, 
		os_major 
from users inner join dbt.zrh_users AS zu using (user_created) 
group by 1,2,3,4,5,6,7,8,9,10

"""

In [103]:
##### Make checks to data consistency.
df = pd.read_csv("kyc_removing_video_link.csv")

# d = df.groupby("user_id")["treatment"].agg("nunique").reset_index()
# if d.loc[d["treatment"]>1,:].shape[0] > 0:
# print("Users were part of both groups")
# else:
# print("No single user saw both variants.")

### Step 3. Sample size


It's not ideal that the sample sizes are quite far apart. I tested the validity of the assignment by checking how often users in the treatment actually clicked the link: https://metabase-product.tech26.de/question/2013?app_version=n26-android_3.45&app_version=n26-ios_3.46&app_version=n26-ios_3.45&start_date=2020-05-20


In [19]:
df = pd.read_csv("kyc_removing_video_link.csv")

df = df.groupby(["treatment"])["user_id"].agg("nunique").reset_index()

alt.Chart(df).mark_bar().encode(
    x=alt.X("treatment:N", axis=alt.Axis(title="Group")),
    y=alt.Y("user_id:Q", axis=alt.Axis(title="Number of users")),
).properties(width=500, height=500, title="Sample size")

alt.Chart(...)

### Step 4. Main KPI 

Our intervention starts at the beginning of the KYC flow - before the KYC process in cmd is officially initiated. 

The goal of this test is to increase the % of users that start the KYC flow and then also successfully manage to initiate a KYC process. 

This % of users that initiate a KYC process is shown below. 

In [59]:
df = pd.read_csv("kyc_removing_video_link.csv")

# make sure we are unique on the user level.
df = (
    df.groupby(["treatment", "user_id"])["kyc_first_initiated"].agg("min").reset_index()
)

# now, count how many users managed to initiate kyc.
df["kyci"] = 0
df.loc[df["kyc_first_initiated"].isna() == False, "kyci"] = 1

df = df.groupby(["treatment"])["kyci"].agg("mean").reset_index()

alt.Chart(df).mark_bar().encode(
    x=alt.X("treatment:N", axis=alt.Axis(title="Group")),
    y=alt.Y(
        "kyci:Q",
        axis=alt.Axis(title="% of users that initiate KYC", format="%"),
        scale=alt.Scale(domain=[0.90, 0.94]),
    ),
).properties(width=500, height=500, title="% of users that initiate KYC")

alt.Chart(...)

There is a difference between the two groups. The next step is to check whether this difference is significant.

Why do we do this? This check helps us understand whether the difference between the groups could have also arisen by chance with a high probability. 

If we claim a "statistically significant difference", the concrete interpretation is that the chance of this difference being created by chance is 5%. 

In [38]:
from statsmodels.stats.proportion import proportions_ztest

df = pd.read_csv("kyc_removing_video_link.csv")

# make sure we are unique on the user level.
df = (
    df.groupby(["treatment", "user_id"])["kyc_first_initiated"].agg("min").reset_index()
)

# now, count how many users managed to initiate kyc.
df["kyci"] = 0
df.loc[df["kyc_first_initiated"].isna() == False, "kyci"] = 1

df = df.groupby("treatment")["kyci"].agg(["count", "sum"]).reset_index()

# run z test. (two sided)
stat, pval = proportions_ztest(df["sum"], df["count"])

print(
    "The z-score for this test is %s which corresponds to a p-value of %s"
    % (round(stat, 2), round(pval, 4))
)

if pval < 0.05:
    print("The difference is significant.")
else:
    print("The difference is not signficiant.")


# compute the effect size.
df["cr"] = df["sum"] / df["count"]

mu_v = df.loc[df["treatment"] == "Treatment", "cr"][1]
mu_c = df.loc[df["treatment"] != "Treatment", "cr"][0]
effect_size = 100 * ((mu_v - mu_c) / mu_c)

print("The effect size is %s Percent" % (round(effect_size, 2)))

The z-score for this test is -2.92 which corresponds to a p-value of 0.0035
The difference is significant.
The effect size is 1.21 Percent


### Step 5. Check whether treatment negatively affect KYC conversion rates. 

Maybe the lack of information prevents some people to complete KYC? This does not seem to be an issue.

We compute the % KYCi to KYCc (7 day) for control and treatment unsers: we see no difference between the two groups.

In [104]:
from datetime import datetime, timedelta

df = pd.read_csv("kyc_removing_video_link.csv")

# make sure we are unique on the user level.
df = (
    df.groupby(["treatment", "user_id"])["kyc_first_completed", "kyc_first_initiated"]
    .agg("min")
    .reset_index()
)

# drop all users that did not initiate kyc
df = df.loc[df["kyc_first_initiated"].isna() == False, :]
# drop recent joiners to the sample
df = df.loc[
    pd.to_datetime(df["kyc_first_initiated"]) <= datetime.now() - timedelta(days=7), :
]

df["date_diff"] = (
    pd.to_datetime(df["kyc_first_completed"])
    - pd.to_datetime(df["kyc_first_initiated"])
).dt.days
# now, count how many users managed to initiate kyc.
df["kycc"] = 0
df.loc[df["date_diff"] <= 7, "kycc"] = 1


df = df.groupby(["treatment"])["kycc"].agg("mean").reset_index()

alt.Chart(df).mark_bar().encode(
    x=alt.X("treatment:N", axis=alt.Axis(title="Group")),
    y=alt.Y("kycc:Q", axis=alt.Axis(title="% of users that complete KYC", format="%")),
).properties(width=500, height=500, title="% of users that complete KYC")

/Users/fabioschmidt-fischbach/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


alt.Chart(...)

In [95]:
df.head()

,treatment,kycc
0,Control,0.601505
1,Treatment,0.603084


In [100]:
from datetime import datetime, timedelta

df = pd.read_csv("kyc_removing_video_link.csv")

# make sure we are unique on the user level.
df = (
    df.groupby(["treatment", "user_id"])["kyc_first_completed", "kyc_first_initiated"]
    .agg("min")
    .reset_index()
)

# now, count how many users managed to initiate kyc.
df["kycc"] = 0
df.loc[df["kyc_first_completed"].isna() == False, "kycc"] = 1

df = df.groupby(["treatment"])["kycc"].agg("mean").reset_index()

alt.Chart(df).mark_bar().encode(
    x=alt.X("treatment:N", axis=alt.Axis(title="Group")),
    y=alt.Y("kycc:Q", axis=alt.Axis(title="% of users that complete KYC", format="%")),
).properties(
    width=500, height=500, title="% of users that complete KYC from flow start"
)

/Users/fabioschmidt-fischbach/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


alt.Chart(...)

In [101]:
df.head()

,treatment,kycc
0,Control,0.631988
1,Treatment,0.640633


### Step 6. Show primary KPI by a couple of other dimensions. 

In [55]:
df = pd.read_csv("kyc_removing_video_link.csv")

# make sure we are unique on the user level.
df = (
    df.groupby(["treatment", "platform", "user_id"])["kyc_first_initiated"]
    .agg("min")
    .reset_index()
)

# now, count how many users managed to initiate kyc.
df["kyci"] = 0
df.loc[df["kyc_first_initiated"].isna() == False, "kyci"] = 1

df = df.groupby(["treatment", "platform"])["kyci"].agg("mean").reset_index()

alt.Chart(df).mark_bar().encode(
    x=alt.X("treatment:N", axis=alt.Axis(title="Group")),
    y=alt.Y("kyci:Q", axis=alt.Axis(title="% of users that initiate KYC", format="%")),
    column="platform",
).properties(width=500, height=500, title="% of users that initiate KYC")

alt.Chart(...)

In [56]:
df = pd.read_csv("kyc_removing_video_link.csv")

# make sure we are unique on the user level.
df = (
    df.groupby(["treatment", "age_group", "user_id"])["kyc_first_initiated"]
    .agg("min")
    .reset_index()
)

# now, count how many users managed to initiate kyc.
df["kyci"] = 0
df.loc[df["kyc_first_initiated"].isna() == False, "kyci"] = 1

df = df.groupby(["treatment", "age_group"])["kyci"].agg("mean").reset_index()

alt.Chart(df).mark_bar().encode(
    x=alt.X("treatment:N", axis=alt.Axis(title="Group")),
    y=alt.Y("kyci:Q", axis=alt.Axis(title="% of users that initiate KYC", format="%")),
    column="age_group",
).properties(width=500, height=500, title="% of users that initiate KYC")

alt.Chart(...)

In [58]:
df = pd.read_csv("kyc_removing_video_link.csv")

# make sure we are unique on the user level.
df = (
    df.groupby(["treatment", "gender", "user_id"])["kyc_first_initiated"]
    .agg("min")
    .reset_index()
)

# now, count how many users managed to initiate kyc.
df["kyci"] = 0
df.loc[df["kyc_first_initiated"].isna() == False, "kyci"] = 1

df = df.groupby(["treatment", "gender"])["kyci"].agg("mean").reset_index()

alt.Chart(df).mark_bar().encode(
    x=alt.X("treatment:N", axis=alt.Axis(title="Group")),
    y=alt.Y("kyci:Q", axis=alt.Axis(title="% of users that initiate KYC", format="%")),
    column="gender",
).properties(width=500, height=500, title="% of users that initiate KYC")

alt.Chart(...)